In [1]:
import json
import pandas as pd
import numpy as np

In [2]:
half = "1H"
max_min = 15
min_min = max_min - 15
max_sec = max_min*60
min_sec = min_min*60
table_name = "final_in_poss_df"

In [3]:
print(f'{table_name}_{half}_{min_min}_{max_min}.csv')

final_in_poss_df_1H_0_15.csv


In [4]:
with open("../../events/events_England.json", "r") as file:
    event_data_2 = json.load(file)

df2 = pd.DataFrame(event_data_2)
df2 = df2[df2.matchPeriod == half]
df2 = df2[df2.eventSec >= min_sec]
if max_min < 45:
    df2 = df2[df2.eventSec < max_sec]

with open("../../events/events_France.json", "r") as file:
    event_data_3 = json.load(file)

df3 = pd.DataFrame(event_data_3)
df3 = df3[df3.matchPeriod == half]
df3 = df3[df3.eventSec >= min_sec]
if max_min < 45:
    df3 = df3[df3.eventSec < max_sec]


with open("../../events/events_Germany.json", "r") as file:
    event_data_4 = json.load(file)

df4 = pd.DataFrame(event_data_4)
df4 = df4[df4.matchPeriod == half]
df4 = df4[df4.eventSec >= min_sec]
if max_min < 45:
    df4 = df4[df4.eventSec < max_sec]


with open("../../events/events_Italy.json", "r") as file:
    event_data_5 = json.load(file)

df5 = pd.DataFrame(event_data_5)
df5 = df5[df5.matchPeriod == half]
df5 = df5[df5.eventSec >= min_sec]
if max_min < 45:
    df5 = df5[df5.eventSec < max_sec]


with open("../../events/events_Spain.json", "r") as file:
    event_data_6 = json.load(file)

df6 = pd.DataFrame(event_data_6)
df6 = df6[df6.matchPeriod == half]
df6 = df6[df6.eventSec >= min_sec]
if max_min < 45:
    df6 = df6[df6.eventSec < max_sec]


with open("../../events/teams.json", "r") as file:
    teams_data = json.load(file)

teams_df = pd.DataFrame(teams_data)

In [5]:
all_events_df = pd.concat([df2, df3, df4, df5, df6], ignore_index=True)

# Feature engineering - Pass DF - Pass Motifs

In [6]:
passes_df = all_events_df[all_events_df['eventName'] == 'Pass']

# Concatenating teamId and matchId to create a unique identifier for each team in each match
passes_df['uniqueTeamId'] = passes_df['teamId'].astype(str) + "---" + passes_df['matchId'].astype(str)

<ipython-input-6-846069bd07cf>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  passes_df['uniqueTeamId'] = passes_df['teamId'].astype(str) + "---" + passes_df['matchId'].astype(str)


In [7]:
# Function to classify pass motifs
def classify_pass_motif(passes):
    players = passes['playerId'].values
    if len(set(players)) == 2:  # ABAB
        return 'ABAB'
    elif len(set(players)) == 3:
        if players[0] == players[2] or players[1] == players[3]:  # ABCA or ABCB
            return 'ABCA' if players[0] == players[2] else 'ABCB'
        else:
            return 'ABCD'  # ABCD for distinct players or other scenarios
    return 'ABCD'

# Apply the classification within a rolling window
def apply_motif_classification(df):
    # Ensure the DataFrame is sorted by event time
    df = df.sort_values(by='eventSec')
    motifs = []
    # Using a rolling window of 4 events
    for i in range(len(df) - 3):
        window = df.iloc[i:i+4]
        motif = classify_pass_motif(window)
        motifs.append((window.iloc[-1]['uniqueTeamId'], motif))
    return pd.DataFrame(motifs, columns=['uniqueTeamId', 'Motif'])

In [8]:
motif_classification_results = passes_df.groupby('uniqueTeamId').apply(apply_motif_classification)

In [9]:
motif_classification_results.reset_index(drop=True, inplace=True)

In [10]:
motif_classification_results

,uniqueTeamId,Motif
0,10531---2499726,ABCD
1,10531---2499726,ABCD
2,10531---2499726,ABCD
3,10531---2499726,ABCD
4,10531---2499726,ABCD
...,...,...
271964,756---2565923,ABCA
271965,756---2565923,ABCD
271966,756---2565923,ABCD
271967,756---2565923,ABCD


In [11]:
motif_counts = motif_classification_results.groupby(['uniqueTeamId', 'Motif']).size().unstack(fill_value=0)

In [12]:
motif_ratios = motif_counts.div(motif_counts.sum(axis=1), axis=0)
# df to be used in clustering

In [13]:
motif_ratios

Motif,ABAB,ABCA,ABCB,ABCD
uniqueTeamId,,,,
10531---2499726,0.000000,0.112903,0.129032,0.758065
10531---2499736,0.000000,0.111111,0.111111,0.777778
10531---2499741,0.030769,0.107692,0.115385,0.746154
10531---2499757,0.033613,0.126050,0.126050,0.714286
10531---2499766,0.000000,0.087719,0.087719,0.824561
...,...,...,...,...
756---2565880,0.075949,0.139241,0.139241,0.645570
756---2565892,0.047619,0.119048,0.095238,0.738095
756---2565904,0.000000,0.108696,0.086957,0.804348


In [14]:
motif_stats = motif_ratios.agg(['mean', 'std']).T

In [15]:
motif_stats

,mean,std
Motif,,
ABAB,0.034721,0.028929
ABCA,0.107509,0.036457
ABCB,0.108804,0.036526
ABCD,0.748966,0.080183


# Feature engineering - Shot DF

In [16]:
shot_df = all_events_df[all_events_df['eventName'] == 'Shot']
shot_df['uniqueTeamId'] = shot_df['teamId'].astype(str) + "---" + shot_df['matchId'].astype(str)

<ipython-input-16-396a5f5d4462>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shot_df['uniqueTeamId'] = shot_df['teamId'].astype(str) + "---" + shot_df['matchId'].astype(str)


In [17]:
import math

def calculate_shot_distance(row):
    if row['positions'] is not None and len(row['positions']) > 0:
        loc = row['positions'][0]
        # Normalize locations
        x = loc['x'] * (120 / 100.0)  # Normalizing the x coordinate
        y = loc['y'] * (80 / 100.0)   # Normalizing the y coordinate
        # Calculate Euclidean distance from (0,0)
        distance = math.sqrt((120-x)**2 + (80-y)**2)
        return distance
    return None

# Apply the function to calculate shot distance and add it to the DataFrame
shot_df['shot_distance'] = shot_df.apply(calculate_shot_distance, axis=1)

<ipython-input-17-103ed8455f10>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shot_df['shot_distance'] = shot_df.apply(calculate_shot_distance, axis=1)


In [18]:
def classify_shot_by_distance(distance):
    if distance <= 38:
        return "Near"
    elif distance <= 52:
        return "Middle Distance"
    else:
        return "Far"

# Assuming shot_df has a 'shot_distance' column calculated as before
shot_df['shot_class'] = shot_df['shot_distance'].apply(classify_shot_by_distance)

# Grouping shots by their class and counting
shot_class_counts = shot_df.groupby('uniqueTeamId')['shot_class'].value_counts().unstack(fill_value=0)
shot_counts = shot_df.groupby('uniqueTeamId')['subEventName'].value_counts().unstack(fill_value=0)

<ipython-input-18-0451b5045683>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shot_df['shot_class'] = shot_df['shot_distance'].apply(classify_shot_by_distance)


In [19]:
shot_df_final = pd.concat([shot_counts, shot_class_counts], axis=1)

In [20]:
shot_df_final

,Shot,Far,Middle Distance,Near
uniqueTeamId,,,,
10531---2499726,1,0,1,0
10531---2499736,2,1,1,0
10531---2499741,2,0,0,2
10531---2499766,1,0,0,1
10531---2499777,1,0,0,1
...,...,...,...,...
756---2565864,1,0,1,0
756---2565870,3,1,2,0
756---2565880,1,0,0,1


In [21]:
shot_df_final.rename(columns = {'Shot':'Total Shots'}, inplace = True) 

shot_df_final["Far"] = shot_df_final["Far"]/shot_df_final["Total Shots"]
shot_df_final["Middle Distance"] = shot_df_final["Middle Distance"]/shot_df_final["Total Shots"]
shot_df_final["Near"] = shot_df_final["Near"]/shot_df_final["Total Shots"]


shot_df_final.rename(columns = {'Near':'Near Shots Ratio', 'Middle Distance':'Middle Distance Shots Ratio', 'Far':'Far Shots Ratio',}, inplace = True) 

In [22]:
shot_df_final

,Total Shots,Far Shots Ratio,Middle Distance Shots Ratio,Near Shots Ratio
uniqueTeamId,,,,
10531---2499726,1,0.000000,1.000000,0.0
10531---2499736,2,0.500000,0.500000,0.0
10531---2499741,2,0.000000,0.000000,1.0
10531---2499766,1,0.000000,0.000000,1.0
10531---2499777,1,0.000000,0.000000,1.0
...,...,...,...,...
756---2565864,1,0.000000,1.000000,0.0
756---2565870,3,0.333333,0.666667,0.0
756---2565880,1,0.000000,0.000000,1.0


# Feature engineering - Pass DF - Pass Types

In [23]:
def determine_zone(position):
    if 'x' in position and 'y' in position:
        x, y = position['x'], position['y']
        zone = int(x // 33.333) + 1
        return zone
    else:
        return None
    
# Apply the function to determine zones for each event
passes_df['Zone'] = passes_df['positions'].apply(lambda pos: determine_zone(pos[0]) if len(pos) > 0 else None)

<ipython-input-23-1dcee42e8a09>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  passes_df['Zone'] = passes_df['positions'].apply(lambda pos: determine_zone(pos[0]) if len(pos) > 0 else None)


In [24]:
passes_df.loc[passes_df['Zone'] == 4, ['Zone']] = 3

In [25]:
passes_df

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id,uniqueTeamId,Zone
0,8,Simple pass,[{'id': 1801}],25413,"[{'y': 49, 'x': 49}, {'y': 78, 'x': 31}]",2499719,Pass,1609,1H,2.758649,85,177959171,1609---2499719,2
1,8,High pass,[{'id': 1801}],370224,"[{'y': 78, 'x': 31}, {'y': 75, 'x': 51}]",2499719,Pass,1609,1H,4.946850,83,177959172,1609---2499719,1
2,8,Head pass,[{'id': 1801}],3319,"[{'y': 75, 'x': 51}, {'y': 71, 'x': 35}]",2499719,Pass,1609,1H,6.542188,82,177959173,1609---2499719,2
3,8,Head pass,[{'id': 1801}],120339,"[{'y': 71, 'x': 35}, {'y': 95, 'x': 41}]",2499719,Pass,1609,1H,8.143395,82,177959174,1609---2499719,2
4,8,Simple pass,[{'id': 1801}],167145,"[{'y': 95, 'x': 41}, {'y': 88, 'x': 72}]",2499719,Pass,1609,1H,10.302366,85,177959175,1609---2499719,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
526216,8,Simple pass,[{'id': 1801}],8278,"[{'y': 9, 'x': 65}, {'y': 3, 'x': 57}]",2565927,Pass,675,1H,891.564347,85,253300841,675---2565927,2
526220,8,Simple pass,[{'id': 1801}],3310,"[{'y': 3, 'x': 61}, {'y': 3, 'x': 45}]",2565927,Pass,675,1H,894.488356,85,253300845,675---2565927,2
526222,8,Simple pass,[{'id': 1801}],3563,"[{'y': 7, 'x': 50}, {'y': 19, 'x': 46}]",2565927,Pass,675,1H,896.658726,85,253300847,675---2565927,2
526223,8,Simple pass,[{'id': 1801}],8287,"[{'y': 19, 'x': 46}, {'y': 9, 'x': 36}]",2565927,Pass,675,1H,896.850123,85,253300848,675---2565927,2


In [26]:
import math

def calculate_pass_details_from_single_column(row):
    if row['positions'] is not None and len(row['positions']) == 2:
        # Extract and normalize start and end locations
        start_loc = row['positions'][0]
        end_loc = row['positions'][1]
        
        start_x = start_loc['x'] * (120 / 100.0)
        start_y = start_loc['y'] * (80 / 100.0)
        end_x = end_loc['x'] * (120 / 100.0)
        end_y = end_loc['y'] * (80 / 100.0)
        
        # Calculate pass length and angle
        delta_x = end_x - start_x
        delta_y = end_y - start_y
        pass_length = math.sqrt(delta_x**2 + delta_y**2)
        pass_angle = math.atan2(delta_y, delta_x)
        
        return pd.Series([pass_length, pass_angle])
    else:
        return pd.Series([None, None])

In [27]:
passes_df[['calculated_pass_length', 'calculated_pass_angle']] = passes_df.apply(calculate_pass_details_from_single_column, axis=1)


<ipython-input-27-53702553e163>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  passes_df[['calculated_pass_length', 'calculated_pass_angle']] = passes_df.apply(calculate_pass_details_from_single_column, axis=1)
<ipython-input-27-53702553e163>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  passes_df[['calculated_pass_length', 'calculated_pass_angle']] = passes_df.apply(calculate_pass_details_from_single_column, axis=1)


In [28]:
def determine_pass_direction(angle):
    # Forward Pass: -π/4 < angle < π/4
    if -np.pi/4 < angle < np.pi/4:
        return "Forward Pass"
    # Backward Pass: 3π/4 < angle <= π or -π <= angle < -3π/4
    elif 3*np.pi/4 < angle <= np.pi or -np.pi <= angle < -3*np.pi/4:
        return "Backward Pass"
    # Side Pass: Otherwise
    else:
        return "Side Pass"

In [29]:
passes_df['pass_direction'] = passes_df['calculated_pass_angle'].apply(determine_pass_direction)

<ipython-input-29-a0b0c4f17a64>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  passes_df['pass_direction'] = passes_df['calculated_pass_angle'].apply(determine_pass_direction)


In [30]:
passes_df

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id,uniqueTeamId,Zone,calculated_pass_length,calculated_pass_angle,pass_direction
0,8,Simple pass,[{'id': 1801}],25413,"[{'y': 49, 'x': 49}, {'y': 78, 'x': 31}]",2499719,Pass,1609,1H,2.758649,85,177959171,1609---2499719,2,31.698580,2.320495,Side Pass
1,8,High pass,[{'id': 1801}],370224,"[{'y': 78, 'x': 31}, {'y': 75, 'x': 51}]",2499719,Pass,1609,1H,4.946850,83,177959172,1609---2499719,1,24.119701,-0.099669,Forward Pass
2,8,Head pass,[{'id': 1801}],3319,"[{'y': 75, 'x': 51}, {'y': 71, 'x': 35}]",2499719,Pass,1609,1H,6.542188,82,177959173,1609---2499719,2,19.464840,-2.976444,Backward Pass
3,8,Head pass,[{'id': 1801}],120339,"[{'y': 71, 'x': 35}, {'y': 95, 'x': 41}]",2499719,Pass,1609,1H,8.143395,82,177959174,1609---2499719,2,20.505609,1.212026,Side Pass
4,8,Simple pass,[{'id': 1801}],167145,"[{'y': 95, 'x': 41}, {'y': 88, 'x': 72}]",2499719,Pass,1609,1H,10.302366,85,177959175,1609---2499719,2,37.619144,-0.149416,Forward Pass
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
526216,8,Simple pass,[{'id': 1801}],8278,"[{'y': 9, 'x': 65}, {'y': 3, 'x': 57}]",2565927,Pass,675,1H,891.564347,85,253300841,675---2565927,2,10.733126,-2.677945,Backward Pass
526220,8,Simple pass,[{'id': 1801}],3310,"[{'y': 3, 'x': 61}, {'y': 3, 'x': 45}]",2565927,Pass,675,1H,894.488356,85,253300845,675---2565927,2,19.200000,3.141593,Backward Pass
526222,8,Simple pass,[{'id': 1801}],3563,"[{'y': 7, 'x': 50}, {'y': 19, 'x': 46}]",2565927,Pass,675,1H,896.658726,85,253300847,675---2565927,2,10.733126,2.034444,Side Pass
526223,8,Simple pass,[{'id': 1801}],8287,"[{'y': 19, 'x': 46}, {'y': 9, 'x': 36}]",2565927,Pass,675,1H,896.850123,85,253300848,675---2565927,2,14.422205,-2.553590,Backward Pass


In [31]:
np.unique(passes_df.subEventName)

array(['Cross', 'Hand pass', 'Head pass', 'High pass', 'Launch',
       'Simple pass', 'Smart pass'], dtype=object)

In [32]:
passes_df

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id,uniqueTeamId,Zone,calculated_pass_length,calculated_pass_angle,pass_direction
0,8,Simple pass,[{'id': 1801}],25413,"[{'y': 49, 'x': 49}, {'y': 78, 'x': 31}]",2499719,Pass,1609,1H,2.758649,85,177959171,1609---2499719,2,31.698580,2.320495,Side Pass
1,8,High pass,[{'id': 1801}],370224,"[{'y': 78, 'x': 31}, {'y': 75, 'x': 51}]",2499719,Pass,1609,1H,4.946850,83,177959172,1609---2499719,1,24.119701,-0.099669,Forward Pass
2,8,Head pass,[{'id': 1801}],3319,"[{'y': 75, 'x': 51}, {'y': 71, 'x': 35}]",2499719,Pass,1609,1H,6.542188,82,177959173,1609---2499719,2,19.464840,-2.976444,Backward Pass
3,8,Head pass,[{'id': 1801}],120339,"[{'y': 71, 'x': 35}, {'y': 95, 'x': 41}]",2499719,Pass,1609,1H,8.143395,82,177959174,1609---2499719,2,20.505609,1.212026,Side Pass
4,8,Simple pass,[{'id': 1801}],167145,"[{'y': 95, 'x': 41}, {'y': 88, 'x': 72}]",2499719,Pass,1609,1H,10.302366,85,177959175,1609---2499719,2,37.619144,-0.149416,Forward Pass
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
526216,8,Simple pass,[{'id': 1801}],8278,"[{'y': 9, 'x': 65}, {'y': 3, 'x': 57}]",2565927,Pass,675,1H,891.564347,85,253300841,675---2565927,2,10.733126,-2.677945,Backward Pass
526220,8,Simple pass,[{'id': 1801}],3310,"[{'y': 3, 'x': 61}, {'y': 3, 'x': 45}]",2565927,Pass,675,1H,894.488356,85,253300845,675---2565927,2,19.200000,3.141593,Backward Pass
526222,8,Simple pass,[{'id': 1801}],3563,"[{'y': 7, 'x': 50}, {'y': 19, 'x': 46}]",2565927,Pass,675,1H,896.658726,85,253300847,675---2565927,2,10.733126,2.034444,Side Pass
526223,8,Simple pass,[{'id': 1801}],8287,"[{'y': 19, 'x': 46}, {'y': 9, 'x': 36}]",2565927,Pass,675,1H,896.850123,85,253300848,675---2565927,2,14.422205,-2.553590,Backward Pass


In [33]:
#passes_count = passes_df.groupby('teamId')['subEventName'].value_counts().unstack(fill_value=0)

In [34]:
# Step 1: Combine 'pass_direction' and 'Zone' into a single column
passes_df['dir_zone'] = passes_df['pass_direction'].astype(str) + '_Zone_' + passes_df['Zone'].astype(str)

# Step 2: Group by 'uniqueTeamId' and the new 'dir_zone' column, then count
direction_zone_count = passes_df.groupby(['uniqueTeamId', 'dir_zone']).size()

# Step 3: Pivot the results to have 'uniqueTeamId' as rows and each unique 'dir_zone' combination as columns
direction_zone_count_pivot = direction_zone_count.unstack(fill_value=0)

direction_count = passes_df.groupby('uniqueTeamId')['pass_direction'].value_counts().unstack(fill_value=0)

pass_direction_df = pd.concat([direction_zone_count_pivot, direction_count], axis=1)

<ipython-input-34-54e3dcb6a0f6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  passes_df['dir_zone'] = passes_df['pass_direction'].astype(str) + '_Zone_' + passes_df['Zone'].astype(str)


In [35]:
pass_direction_df

,Backward Pass_Zone_1,Backward Pass_Zone_2,Backward Pass_Zone_3,Forward Pass_Zone_1,Forward Pass_Zone_2,Forward Pass_Zone_3,Side Pass_Zone_1,Side Pass_Zone_2,Side Pass_Zone_3,Backward Pass,Forward Pass,Side Pass
uniqueTeamId,,,,,,,,,,,,
10531---2499726,2,8,3,7,13,1,13,13,5,13,21,31
10531---2499736,2,4,1,4,8,3,9,8,9,7,15,26
10531---2499741,2,18,3,12,19,2,25,48,4,23,33,77
10531---2499757,1,13,4,24,14,5,24,25,12,18,43,61
10531---2499766,0,8,2,9,9,0,9,19,4,10,18,32
...,...,...,...,...,...,...,...,...,...,...,...,...
756---2565880,3,8,2,11,9,2,17,22,8,13,22,47
756---2565892,2,5,1,9,10,1,8,8,1,8,20,17
756---2565904,3,2,2,13,7,0,14,7,1,7,20,22


## Pass Types

In [39]:
high_pass_types = ['High pass', 'Cross', 'Launch']

all_pass_types = passes_df['subEventName'].unique()
low_pass_types = [pt for pt in all_pass_types if pt not in high_pass_types and pt != 'Smart pass']

# --- overall ---
pass_counts = passes_df.groupby(['uniqueTeamId', 'subEventName']).size().unstack(fill_value=0)
pass_counts['Total_Passes'] = pass_counts.sum(axis=1)

# reindex to avoid missing columns (robust)
pass_counts['High_pass_Total'] = pass_counts.reindex(columns=high_pass_types, fill_value=0).sum(axis=1)
pass_counts['Low_pass_Total']  = pass_counts.reindex(columns=low_pass_types,  fill_value=0).sum(axis=1)

pass_counts['High_pass_Ratio_Total'] = pass_counts['High_pass_Total'] / pass_counts['Total_Passes']
pass_counts['Low_pass_Ratio_Total']  = pass_counts['Low_pass_Total'] / pass_counts['Total_Passes']

final_pass_types_df = pass_counts[['Total_Passes', 'High_pass_Ratio_Total', 'Low_pass_Ratio_Total']].copy()

# --- zone-specific ---
zones = passes_df['Zone'].unique()
for zone in zones:
    zone_df = passes_df[passes_df['Zone'] == zone]

    zone_pass_counts = zone_df.groupby(['uniqueTeamId', 'subEventName']).size().unstack(fill_value=0)
    zone_pass_counts['Total_Passes'] = zone_pass_counts.sum(axis=1)

    zone_high_total = zone_pass_counts.reindex(columns=high_pass_types, fill_value=0).sum(axis=1)
    zone_low_total  = zone_pass_counts.reindex(columns=low_pass_types,  fill_value=0).sum(axis=1)

    # avoid divide-by-zero: teams with 0 passes in that zone -> NaN
    denom = zone_pass_counts['Total_Passes'].replace(0, pd.NA)

    final_pass_types_df[f'High_pass_Ratio_Zone{zone}'] = zone_high_total / denom
    final_pass_types_df[f'Low_pass_Ratio_Zone{zone}']  = zone_low_total / denom


In [41]:
final_pass_types_df

subEventName,Total_Passes,High_pass_Ratio_Total,Low_pass_Ratio_Total,High_pass_Ratio_Zone2,Low_pass_Ratio_Zone2,High_pass_Ratio_Zone1,Low_pass_Ratio_Zone1,High_pass_Ratio_Zone3,Low_pass_Ratio_Zone3
uniqueTeamId,,,,,,,,,
10531---2499726,65,0.184615,0.815385,0.088235,0.911765,0.227273,0.772727,0.444444,0.555556
10531---2499736,48,0.145833,0.854167,0.050000,0.950000,0.266667,0.733333,0.153846,0.846154
10531---2499741,133,0.090226,0.902256,0.094118,0.894118,0.076923,0.923077,0.111111,0.888889
10531---2499757,122,0.073770,0.901639,0.076923,0.884615,0.061224,0.938776,0.095238,0.857143
10531---2499766,60,0.183333,0.800000,0.083333,0.888889,0.388889,0.611111,0.166667,0.833333
...,...,...,...,...,...,...,...,...,...
756---2565880,82,0.243902,0.756098,0.153846,0.846154,0.193548,0.806452,0.666667,0.333333
756---2565892,45,0.222222,0.777778,0.130435,0.869565,0.263158,0.736842,0.666667,0.333333
756---2565904,49,0.326531,0.673469,0.312500,0.687500,0.333333,0.666667,0.333333,0.666667


# Pass_Connectivity

In [42]:
from collections import defaultdict
import networkx as nx

In [43]:
def calculate_pass_connectivity(group):
    # Initialize two empty graphs for the two teams in the match
    team_graphs = {team_id: nx.Graph() for team_id in group['uniqueTeamId'].unique()}
    
    # Track the previous player and team to identify consecutive passes
    prev_player = None
    prev_team = None
    
    for i, pass_ in group.iterrows():
        current_player = pass_['playerId']
        current_team = pass_['uniqueTeamId']
        
        # Check for consecutive passes within the same team
        if prev_team == current_team and prev_player is not None:
            team_graphs[current_team].add_edge(prev_player, current_player)
        
        # Update the previous player and team for the next iteration
        prev_player = current_player
        prev_team = current_team
    
    # Calculate connectivity for each team graph
    connectivities = {}
    for team_id, graph in team_graphs.items():
        if len(graph) == 0:  # Handle cases where a team made no passes or only incomplete passes
            connectivity = 0
        else:
            connectivity = nx.average_node_connectivity(graph)
        connectivities[team_id] = connectivity
    
    return connectivities

In [44]:
test_dfdf = passes_df.iloc[:1000]

In [45]:
connectivity_results = [calculate_pass_connectivity(group) for _, group in passes_df.groupby(['matchId'])]

In [46]:
connectivity_results

[{'1609---2499719': 3.7818181818181817, '1631---2499719': 2.2},
 {'1625---2499720': 5.5636363636363635, '1651---2499720': 2.4909090909090907},
 {'1646---2499721': 1.7857142857142858, '1610---2499721': 4.672727272727273},
 {'1628---2499722': 2.981818181818182, '1673---2499722': 2.6},
 {'1623---2499723': 2.5272727272727273, '1639---2499723': 2.6666666666666665},
 {'1611---2499724': 4.418181818181818, '1633---2499724': 2.911111111111111},
 {'1613---2499725': 2.7818181818181817, '1624---2499725': 4.090909090909091},
 {'10531---2499726': 5.488888888888889, '1619---2499726': 3.2222222222222223},
 {'1644---2499727': 2.7454545454545456, '1612---2499727': 3.290909090909091},
 {'1627---2499728': 3.036363636363636, '1659---2499728': 3.688888888888889},
 {'1644---2499729': 2.2545454545454544, '1659---2499729': 4.163636363636364},
 {'1627---2499730': 1.4363636363636363, '1646---2499730': 5.155555555555556},
 {'1673---2499731': 3.963636363636364, '1613---2499731': 2.290909090909091},
 {'1631---24997

In [47]:
connectivity_df = [(k, v) for d in connectivity_results for k, v in d.items()]
connectivity_df = pd.DataFrame(connectivity_df, columns=['uniqueTeamId', 'Connectivity'])

In [48]:
connectivity_df.set_index('uniqueTeamId', inplace=True)
connectivity_df

,Connectivity
uniqueTeamId,
1609---2499719,3.781818
1631---2499719,2.200000
1625---2499720,5.563636
1651---2499720,2.490909
1646---2499721,1.785714
...,...
3158---2576336,4.327273
3164---2576337,3.727273
3204---2576337,1.800000


In [49]:
final_df = pd.concat([motif_ratios, final_pass_types_df, pass_direction_df, shot_df_final, connectivity_df], axis=1)
# Iterate through each column in final_df to find and adjust the specified columns
for col in final_df.columns:
    # Check if the column name contains 'Backward', 'Forward', or 'Side'
    if any(keyword in col for keyword in ['Backward', 'Forward', 'Side']):
        # Divide the column by the 'Total_Passes' column
        final_df[col] = final_df[col] / final_df['Total_Passes']
        # Rename the column to add " Ratio" at the end
        final_df.rename(columns={col: col + " Ratio"}, inplace=True)

In [50]:
final_df

,ABAB,ABCA,ABCB,ABCD,Total_Passes,High_pass_Ratio_Total,Low_pass_Ratio_Total,High_pass_Ratio_Zone2,Low_pass_Ratio_Zone2,High_pass_Ratio_Zone1,...,Side Pass_Zone_2 Ratio,Side Pass_Zone_3 Ratio,Backward Pass Ratio,Forward Pass Ratio,Side Pass Ratio,Total Shots,Far Shots Ratio,Middle Distance Shots Ratio,Near Shots Ratio,Connectivity
uniqueTeamId,,,,,,,,,,,,,,,,,,,,,
10531---2499726,0.000000,0.112903,0.129032,0.758065,65,0.184615,0.815385,0.088235,0.911765,0.227273,...,0.200000,0.076923,0.200000,0.323077,0.476923,1.0,0.00,1.00,0.0,5.488889
10531---2499736,0.000000,0.111111,0.111111,0.777778,48,0.145833,0.854167,0.050000,0.950000,0.266667,...,0.166667,0.187500,0.145833,0.312500,0.541667,2.0,0.50,0.50,0.0,3.290909
10531---2499741,0.030769,0.107692,0.115385,0.746154,133,0.090226,0.902256,0.094118,0.894118,0.076923,...,0.360902,0.030075,0.172932,0.248120,0.578947,2.0,0.00,0.00,1.0,4.963636
10531---2499757,0.033613,0.126050,0.126050,0.714286,122,0.073770,0.901639,0.076923,0.884615,0.061224,...,0.204918,0.098361,0.147541,0.352459,0.500000,NaN,NaN,NaN,NaN,4.654545
10531---2499766,0.000000,0.087719,0.087719,0.824561,60,0.183333,0.800000,0.083333,0.888889,0.388889,...,0.316667,0.066667,0.166667,0.300000,0.533333,1.0,0.00,0.00,1.0,3.400000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
756---2565880,0.075949,0.139241,0.139241,0.645570,82,0.243902,0.756098,0.153846,0.846154,0.193548,...,0.268293,0.097561,0.158537,0.268293,0.573171,1.0,0.00,0.00,1.0,3.618182
756---2565892,0.047619,0.119048,0.095238,0.738095,45,0.222222,0.777778,0.130435,0.869565,0.263158,...,0.177778,0.022222,0.177778,0.444444,0.377778,2.0,0.00,0.50,0.5,4.305556
756---2565904,0.000000,0.108696,0.086957,0.804348,49,0.326531,0.673469,0.312500,0.687500,0.333333,...,0.142857,0.020408,0.142857,0.408163,0.448980,NaN,NaN,NaN,NaN,3.254545


In [51]:
table_name = "final_in_poss_df"

if half is not None:
    final_df.to_csv(f'{table_name}_{half}_{min_min}_{max_min}.csv', index=True)
else:
    final_df.to_csv(f'{table_name}.csv', index=True)

In [54]:
f"{table_name}_{half}_{min_min}_{max_min}"

'final_in_poss_df_1H_0_15'

# END OF FILE